In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Artificial Neural Networks

#install theano
!pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git
    
#keras and tensorflow are already installed

In [ ]:
#import dataset
dataset = pd.read_csv('/kaggle/input/churn-modelling/Churn_Modelling.csv')
dataset

In [ ]:
#Create matrix of independent variables
X = dataset.iloc[:,3:13].values #[:,1] would return an array but we want X to be a 
#matrix

#Create array of dependent variable
y = dataset.iloc[:,13].values

#Encoding categorical independent variable(s)
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer

#Encode Country and Gender
label_encoder_X1 = LabelEncoder()
X[:, 1] = label_encoder_X1.fit_transform(X[:, 1])
label_encoder_X2 = LabelEncoder()
X[:, 2] = label_encoder_X2.fit_transform(X[:, 2])

#Transform Country into 3 columns
ct = ColumnTransformer(
    [('oh_enc',OneHotEncoder(sparse=False),[1]),],
     remainder='passthrough'
     )
X = ct.fit_transform(X)

#Avoid dummy variable trap
X = X[:,1:]


In [ ]:
#Splitting into test set and training set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

#feature scaling
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

#Build ANN

#import keras
import keras
from keras.models import Sequential
from keras.layers import Dense

#initializing the ANN
classifier = Sequential()

#adding the input layer and the first hidden layer
#tip output_dim = average of nodes in input layers and nodes in output layers (11+1)/ 2 = 6
classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu', 
                     input_shape=(11,)))

#adding the second hidden layer
classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu'))

#adding the output layer
classifier.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))

#compiling the ANN - adam(stochastic gradient descent) - since output layer is using
#sigmoid function, we will use binary_crossentropy - if dependent variable > 2 categories,
#then loss='categorical_crossentropy'
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#fitting ANN to the training set
classifier.fit(X_train, y_train, batch_size=10, epochs=100)

In [ ]:
#Predicting the test set value
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

#Making the confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

In [ ]:
#accuracy
(cm[0,0]+cm[1,1])/(cm[0,0]+cm[0,1]+cm[1,0]+cm[1,1])

In [ ]:
classifier.summary()